# Tests for Face detection with OpenCV
the following notebook contains some test with different model/images for face detection. 

In [26]:
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt
print("OpenCV: "+ str(cv2.__version__))
print("IMPORTS OK")

OpenCV: 3.4.5
IMPORTS OK


In [27]:
image_path = "./img/faces.png"
#show preview image in notebook
show_image =False
WIDTH_ROBOT = 960
HEIGHT_ROBOT = 600

In [28]:
#Read image
img = cv2.imread(image_path,cv2.IMREAD_COLOR )
#calculate h and w
(h, w) = img.shape[:2]
WIDTH = w
HEIGHT = h
print("dimensions image upload: "+ str((h,w)) )
print("dimensions robot camera: "+ str((HEIGHT_ROBOT, WIDTH_ROBOT)) )
print("Same dimensions: "+str((h,w)==(HEIGHT, WIDTH)))

if show_image:
    #cv2.imshow("image",img) #not working in Jupiter notebook
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

dimensions image upload: (600, 960)
dimensions robot camera: (600, 960)
Same dimensions: True


## HAAR CLASSIFIER FACE DETECTION

In [29]:
#Load Classifier
CASC_PATH = "./models/haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(CASC_PATH)
print("Classifier Loaded")

Classifier Loaded


In [30]:
img_haar = img.copy() #copy img

if show_image:
    print("BEFORE:")
    plt.imshow(img_haar)
    plt.show()

#FORWARD
faces = face_cascade.detectMultiScale(img_haar, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#print("faces detected: ",faces.shape[0])
print("Detected: "+str(len(faces)))

#draw bounding boxes
for face in faces:
    x, y, w, h = face
    cv2.rectangle(img_haar, (x, y), (x+w, y+h), (0, 255, 0), 2)

if show_image:    
    print("RESULT:")
    plt.imshow(img_haar)
    plt.show()

#SAVE
cv2.imwrite("./img/save.png",img_haar)

Detected: 15


True

## DLIB

In [31]:
import cv2
import dlib  #requires pip install dlib
detector = dlib.get_frontal_face_detector()

img_dlib = img.copy() #copy img

if show_image:
    print("BEFORE:")
    plt.imshow(img_dlib)
    plt.show()

#FORWARD
gray = cv2.cvtColor(img_dlib, cv2.COLOR_BGR2GRAY)
faces = detector(gray, 1) # result
print("Detected: "+str(len(faces)))

#to draw faces on image
for result in faces:
    x1 = result.left()
    y1 = result.top()
    x2 = result.right()
    y2 = result.bottom()
    cv2.rectangle(img_dlib, (x1, y1), (x2, y2), (0,255 , 0), 2)
    
if show_image:    
    print("RESULT:")
    plt.imshow(img_dlib)
    plt.show()
    
#SAVE
cv2.imwrite("./img/save.png",img_dlib)

Detected: 18


True

## DNN FACE DETECTION

In [32]:
#Load net
PROTO = "./models/res10_deploy.prototxt"
MODEL = "./models/res10_300x300_ssd_iter_140000_fp16.caffemodel"
net = cv2.dnn.readNetFromCaffe(PROTO, MODEL)
print("Net Loaded")

Net Loaded


In [33]:
img_dnn = img.copy() #copy img

if show_image:
    print("BEFORE:")
    plt.imshow(img_dnn)
    plt.show()

#FORWARD image
#blob = cv2.dnn.blobFromImage(cv2.resize(img_dnn, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
blob = cv2.dnn.blobFromImage(img_dnn, 1.0, (300, 300), [104, 117, 123], False, False)
net.setInput(blob)
detections = net.forward()
print("Detected: " + str(detections.shape[2]))
detected_after_threshold=0

#draw bounding box
conf_threshold=0.15
bboxes = []
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > conf_threshold:
        detected_after_threshold+=1
        x1 = int(detections[0, 0, i, 3] * WIDTH)
        y1 = int(detections[0, 0, i, 4] * HEIGHT)
        x2 = int(detections[0, 0, i, 5] * WIDTH)
        y2 = int(detections[0, 0, i, 6] * HEIGHT)
        cv2.rectangle(img_dnn, (x1, y1), (x2, y2), (0, 255, 0), 2)
print("Detected: " + str(detected_after_threshold) + " (after threshold)")

if show_image:
    print("RESULT:")       
    plt.imshow(img_dnn)
    plt.show()

#SAVE
cv2.imwrite("./img/save.png",img_dnn)

Detected: 189
Detected: 14 (after threshold)


True

## OpenCV FaceDetector

In [34]:
#Load net
CONFIG = "./models/opencv_face_detector.pbtxt"
MODEL = "./models/opencv_face_detector_uint8.pb"
net = cv2.dnn.readNetFromTensorflow(MODEL, CONFIG)
print("Net Loaded")

Net Loaded


In [35]:
img_cvdet = img.copy() #copy img

if show_image:
    print("BEFORE:")
    plt.imshow(img_cvdet)
    plt.show()

#FORWARD image
blob = cv2.dnn.blobFromImage(img_cvdet, 1.0, (300, 300), [104, 117, 123], True, False)
net.setInput(blob)
detections = net.forward()
print("Detected: " + str(detections.shape[2]))
detected_after_threshold=0

#draw bounding box
conf_threshold=0.15
bboxes = []
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > conf_threshold:
        detected_after_threshold+=1
        x1 = int(detections[0, 0, i, 3] * WIDTH)
        y1 = int(detections[0, 0, i, 4] * HEIGHT)
        x2 = int(detections[0, 0, i, 5] * WIDTH)
        y2 = int(detections[0, 0, i, 6] * HEIGHT)
        bboxes.append([x1, y1, x2, y2])
        cv2.rectangle(img_cvdet, (x1, y1), (x2, y2), (0, 255, 0), 2)
print("Detected: " + str(detected_after_threshold) + " (after threshold)")

if show_image:
    print("RESULT:")       
    plt.imshow(img_cvdet)
    plt.show()

#SAVE
cv2.imwrite("./img/save.png",img_cvdet)

Detected: 200
Detected: 1 (after threshold)


True